In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# Load training dataset
df = pd.read_csv('result.csv')

df.head(5)

,Unnamed: 0,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_chanceCreationShootingClass,away_chanceCreationPositioningClass,away_defencePressure,away_defencePressureClass,away_defenceAggression,away_defenceAggressionClass,away_defenceTeamWidth,away_defenceTeamWidthClass,away_defenceDefenderLineClass,capacity
0,0,1729,1729,2008,1,2008-08-17,489042,10260,10261,1,...,Normal,Organised,45,Medium,65,Press,45,Normal,Cover,75811
1,1,1729,1729,2008,1,2008-08-16,489043,9825,8659,1,...,Normal,Organised,70,High,70,Double,70,Wide,Cover,60361
2,2,1729,1729,2008,1,2008-08-16,489044,8472,8650,0,...,Lots,Organised,40,Medium,60,Press,40,Normal,Cover,48707
3,3,1729,1729,2008,1,2008-08-16,489045,8654,8528,2,...,Normal,Organised,35,Medium,70,Double,35,Normal,Cover,35303
4,4,1729,1729,2008,1,2008-08-16,489047,8668,8655,2,...,Normal,Organised,45,Medium,70,Double,45,Normal,Cover,40157


In [3]:
# pre process data
# 1. One hot encoding
df = pd.get_dummies(df)

df.head(5)

,Unnamed: 0,country_id,league_id,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,away_chanceCreationPositioningClass_Organised,away_defencePressureClass_Deep,away_defencePressureClass_High,away_defencePressureClass_Medium,away_defenceAggressionClass_Double,away_defenceAggressionClass_Press,away_defenceTeamWidthClass_Narrow,away_defenceTeamWidthClass_Normal,away_defenceTeamWidthClass_Wide,away_defenceDefenderLineClass_Cover
0,0,1729,1729,2008,1,489042,10260,10261,1,1,...,1,0,0,1,0,1,0,1,0,1
1,1,1729,1729,2008,1,489043,9825,8659,1,0,...,1,0,1,0,1,0,0,0,1,1
2,2,1729,1729,2008,1,489044,8472,8650,0,1,...,1,0,0,1,0,1,0,1,0,1
3,3,1729,1729,2008,1,489045,8654,8528,2,1,...,1,0,0,1,1,0,0,1,0,1
4,4,1729,1729,2008,1,489047,8668,8655,2,3,...,1,0,0,1,1,0,0,1,0,1


In [4]:
# split into training and testing dataset

feature_names = df.columns[1:].tolist()
feature_names.remove('match_result')

# Removing player ids
for col in [f'home_player_{i}' for i in range(1, 12)]:
    feature_names.remove(col)
for col in [f'away_player_{i}' for i in range(1, 12)]:
    feature_names.remove(col)

X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df['match_result'], test_size=0.2, random_state=123456)

In [5]:
# Parameters
learning_rate = 0.05
batch_size = 128
display_step = 3

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = len(X_train.columns) # MNIST data input (img shape: 28*28)
num_classes = 3 # home_win, home_lose, draw

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [6]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [8]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [9]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for bid in list(range(X_train.shape[0]//batch_size)):
        batch_x = X_train[bid*batch_size: (bid+1)*batch_size]
        batch_y = y_train[bid*batch_size: (bid+1)*batch_size]

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: pd.get_dummies(batch_y)})
        if bid % display_step == 0 or bid == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: pd.get_dummies(batch_y)})
            print(f"Step {bid}, Minibatch Loss= {loss:.4f} , Training Accuracy= {acc:.3f}")

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test, Y: pd.get_dummies(y_test)}))

Step 0, Minibatch Loss= 547170368.0000 , Training Accuracy= 0.500
Step 1, Minibatch Loss= 1044325312.0000 , Training Accuracy= 0.281
Step 3, Minibatch Loss= 1128715776.0000 , Training Accuracy= 0.438
Step 6, Minibatch Loss= 546161728.0000 , Training Accuracy= 0.422
Optimization Finished!
Testing Accuracy: 0.526087
